# Style transfer

#### Setup

In [1]:
import os

try:
    from google.colab import drive
    drive.mount('/content/drive')

    # Change the directory
    os.chdir('/content/drive/MyDrive/ColabNotebooks/style-transfer')
    is_colab = True
    print("Running on Colab")

except:
    is_colab = False
    print("Running locally")

Running locally


In [2]:
# 35s
if is_colab:
    ! pip install pretty_midi
    # ! apt-get install musescore
    ! pip install dfply

    # ! pip install pyfluidsynth
    # ! apt install fluidsynth

    # !cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2

In [ ]:
#3s
import random
random.seed(42)

import glob
import pandas as pd
from utils.files_utils  import save_pickle, load_pickle, data_path, data_tests_path
from utils.plots_utils import plot_train
from model.train import train_model
from model.colab_tension_vae.params import init
from model.embeddings import obtain_embeddings, get_embeddings_roll_df
from preprocessing import preprocess_data
from tensorflow import keras

## Parámetros

In [3]:
#@title Parámetros ajustables

#@markdown Seleccionar directorios de los datasets
dataset_name = "sonata15_mapleleaf_ds" #@param {type:"string"}
model_name = "urania-test" #@param {type:"string"}
config_name = "8bar" #@param ["8bar", "4bar"]
init(config_name)

#@markdown ¿Entrenamos un nuevo modelo?
if_train = True #@param {type:"boolean"}
entrenar_nuevo = True #@param {type:"boolean"}

#@markdown ¿Cuántas épocas? ¿Cada cuánto grabamos?
epochs =  10#@param {type:"integer"}
checkpt =  1#@param {type:"integer"}

#@markdown Datasets:
dataset1 = "Bach/" #@param {type:"string"}
dataset2 = "ragtime/" #@param {type:"string"}
dataset3 = "Mozart/" #@param {type:"string"}
dataset4 = "Frescobaldi/" #@param {type:"string"}


songs = {
    dataset1[:-1]: [data_path+dataset1+path for path in os.listdir(data_path+dataset1)],
    dataset2[:-1]: [data_path+dataset2+path for path in os.listdir(data_path+dataset2)],
    dataset3[:-1]: [data_path+dataset3+path for path in os.listdir(data_path+dataset3)],
    dataset4[:-1]: [data_path+dataset4+path for path in os.listdir(data_path+dataset4)],
}

In [ ]:
#@title Transformar estilo
ds_original = "Bach" #@param ["Bach", "ragtime", "Mozart", "Frescobaldi"]
ds_objetivo = "Mozart" #@param ["Bach", "ragtime", "Mozart", "Frescobaldi"]

nombre_pickle = ds_original+"2"+ds_objetivo

____
# Preprocessing and Model

Se lo lleva al formato lista de:

- $n$ canciones = lista de:
- 2 voces = lista de:
- $e$ eventos (notas, silencios, etc.)


Armamos el DataFrame con el que manejamos las canciones. Cada fila es una canción y las columnas son:
- `Autor`
- `Título`
- `Id roll`: qué matriz es de la canción
- `Roll` objeto (GuoRoll) con:
    - `matrix`: matriz de $16*n \times 89$ con $n=$ la cantidad de compases
    - `bars`: cantidad de compases por matriz (es el mismo para todo el dataset, con lo cual, podría eliminarse la redundancia en un trabajo futuro)
    - `song`: referencia al objeto `song` al que pertenece si fue obtenido de ahí. Si fue obtenido desde un embedding, es `None` (en un trabajo futuro podría cambiárselo por un singleton).
    - `score`: partitura obtenida a partir de la matriz
    - `midi`: Pretty MIDI obtenido a partir de la matriz

Luego le agregaremos:
- `Embedding`: el vector resultado de encodear el roll

## Preprocessing

In [4]:
try:
    df_preprocessed = load_pickle(name=dataset_name, path=data_path+"preprocessed_data/")
except:
    print("Processing dataset")
    df_preprocessed = preprocess_data(songs)
    save_pickle(df_preprocessed, name=dataset_name, path=data_path+"preprocessed_data/")

## Model

In [ ]:
if if_train:
    vae = train_model(
        df=df_preprocessed,
        model_name=model_name,
        new_training=entrenar_nuevo,
        final_epoch=epochs,
        ckpt=checkpt
    )
else:
    vae = keras.models.load_model(data_path + f"saved_models/{model_name}/")

In [ ]:
glob.glob(rf"{data_path}logs/{model_name}_*.csv")

In [ ]:
id_file =  790#@param {type:"integer"}

callbacks = pd.read_csv(data_path + f"logs/{model_name}_{id_file}.csv")
plot_train(callbacks, epochs)

## Reconstruction

In [ ]:
# TODO: Llevar estas 2 celdas a una única función
try:
    df_emb = load_pickle(name=model_name, path=data_path + "embeddings/")
except:
    df_emb = obtain_embeddings(df_preprocessed, vae)
    save_pickle(df_emb, model_name, data_path + "embeddings/")


In [ ]:
try:
    df_reconstructed = load_pickle(model_name + "recons", data_path + "embeddings/")
except:
    df_reconstructed = get_embeddings_roll_df(df_emb, vae)
    save_pickle(df_reconstructed, model_name + "recons", data_path + "embeddings/")

if is_colab:
  ! sudo apt install lilypond

In [ ]:
for i, r in df_reconstructed.head().iterrows():
    print(r.roll.song.nombre)
    print("Original:")
    r.roll.display_score()
    print("Reconstrucción:")
    r.EmbeddingRoll.display_score()
    print("----------------------------------------------------------------------")

# Working space

In [ ]:
r = load_pickle("roll_8bar_w_rest", data_tests_path)
r.song.nombre

In [5]:
df_ex = df_preprocessed.head().roll[0].matrix

In [6]:
df_ex = pd.DataFrame(df_ex)

In [7]:
df_ex_roll = df_preprocessed.roll[100]

In [8]:
m, mc, b, bc = df_ex_roll.get_melody(), df_ex_roll.get_melody_changes(), df_ex_roll.get_bass(), df_ex_roll.get_bass_changes()

In [ ]:
from utils.files_utils import data_tests_path

save_pickle(df_ex_roll, 'roll_8bar_w_rest', data_tests_path)

In [16]:
import numpy as np

m1bar = np.loadtxt(data_tests_path + "matrix_1bar.csv", delimiter=",", dtype=int)

In [ ]:
! git status

In [ ]:
!git config --global user.email "lsomacal@gmail.com"
!git config --global user.name "LucasSom"

In [ ]:
! git add style-transfer.ipynb model/train.py

In [ ]:
# !git commit -m "Running on colab without reload model"
! git push

In [ ]:
from tensorflow import keras
from model.colab_tension_vae import build_model

vae = keras.models.load_model(data_path + f"saved_models/{model_name}/", custom_objects=dict(kl_beta=build_model.kl_beta))

In [ ]:
vae.summary()